# Práctica del tema 2
Blanca Cano Camarero

In [1]:
# import necesarios para la práctica
import sympy as sp
import numpy as np

## ficheros propios
#### De vuelve el polinomio de interpolación de newton 
import polinomioNewton as pn # polinomioNewton([a-h,a,a+h])

# Ejercicio 1
1.- Obtenga mediante interpolación en el espacio $\mathbb{P}_2$ una fórmula para aproximar $f''(a)$ del tipo combinación
de $f(a-h)$, $f(a)$ y $f(a+h)$.


In [3]:
a,h,z = sp.symbols('a,h,z')
x = [a-h, a, a+h]

#llamamos a nuestra funcioncilla que nos calcula polinomios de interpolación de newton
## (para ver su código mirar en polinomioNewton.py)
p = pn.polinomioNewton(x)

## Finalemente, derivamos pa obtener nuestra fórmula buscada
formula=sp.diff(p,z,2).subs({z:a}).simplify() # esta será la fórmula
# obtenida esta vez para aproximar la derivada segunda f''(a)
formula

(-2*f(a) + f(a - h) + f(a + h))/h**2

## Ejercicio 2
2.- Con la fórmula obtenida en el ejercicio 1, halle una tabla de aproximaciones y errores de $f_1''(2.5)$, siendo $f_1(x)=x^x$, para $h=10^{-i},\; i=1,\ldots,5.$

3.- Sea $f_2(x)=\frac{x^2+40}{x+\sqrt{5x}+7}$. Calcule una tabla que recoja las derivadas de $f_2$ en $x_i=1,2,\ldots,10$, utilizando alguna de las fórmulas de derivación numérica de primer orden obtenidas al inicio de la práctica, con $h=10^{-3}$, y muestre al mismo tiempo el error cometido en cada punto. Repita el ejercicio con la fórmula centrada obtenida para la derivada primera y, finalmente, para la obtenida en el ejercicio 1 (con respecto a la segunda derivada).

In [5]:


# declaramos las fórmulas
def rectangulo_izquierdo(f,a,b):
    return f(a)*(b-a)

def simpson(f, a, b):
    return 1/6*(f(a)+4*f((a+b)/2)+f(b))*(b-a)

def trapecio(f,a,b):
    return (f(a)+f(b))/2*(b-a)

def formula_compuesta (f,formula,a,b,n):
    h = (b-a)/n
    return sum([formula(f,a+i*h,a+(i+1)*h) for i in range(n)])

# Ejericicios 4 y 5

4.- Divida el intervalo $[1,2]$ en 100 partes iguales y aplique las fórmulas del rectángulo, Simpson y trapecio compuestas para aproximar la integral en dicho intervalo de $f_1$. Compare dichos resultados.

5.- Repita el ejercicio 4 para $f_2$. 

In [18]:
def f1(x):
    return x**x
print("___Integral para f1___")
print(f'Integra por rectángulo izquierdo {formula_compuesta(f1,rectangulo_izquierdo,1,2,100)}')
print(f'Integra por trapecio {formula_compuesta(f1,trapecio,1,2,100)}')
print(f'Integra por simpson {formula_compuesta(f1,simpson,1,2,100)}')

def f2(x):
    return (x*x+40)/(x+sqrt(5*x)+7)
print("___Integral para f2___")
print(f'Integra por rectángulo izquierdo {formula_compuesta(f2,rectangulo_izquierdo,1,2,100)}')
print(f'Integra por trapecio {formula_compuesta(f2,trapecio,1,2,100)}')
print(f'Integra por simpson {formula_compuesta(f2,simpson,1,2,100)}')


___Integral para f1___
Integra por rectángulo izquierdo 2.0354943390855578
Integra por trapecio 2.0504943390855574
Integra por simpson 2.0504462346235286
___Integral para f2___
Integra por rectángulo izquierdo 3.77852320278209
Integra por trapecio 3.77658469845732
Integra por simpson 3.77658111777024


Comentario: 
Como podemos observar dan un resultado similar, siend trapecio y simpson muy parecidas entre sí.  

6.- Sea $f_3(x)=x^{15} e^x$ en $[0,2]$. Vamos a dividir el intervalo en $10\times 2^n$ subintervalos, es decir, $10,\,20,\,40,\, 80,\ldots $ y a aplicar la fórmula de Simpson compuesta hasta que la diferencia entre dos aproximaciones consecutivas (por ejemplo, podrían
ser con $20$ y $40$ subintervalos) sea menor que $10^{-2}$, dando en tal caso por buena la última aproximación obtenida. Programe
y calcule dicha aproximación. Compare ambas aproximaciones con
el valor exacto.

In [25]:
import numpy as np
def f3(x):
    return x**(15) * np.exp(x)

## cálculo de valor exacto por regla de barrow
x = sp.symbols("x")
f3s = x**15 * sp.exp(x) 
If3s = sp.integrate(f3s,x)
exacto = If3s.subs(x,2)-If3s.subs(x,0)
print(f'Aplicando la regla de barrow el valor es {exacto.evalf()}')
diferencia = 9
y = formula_compuesta(f3,simpson,0,2,10)
n= 1
while diferencia > 10**(-2):
    ys = formula_compuesta(f3,simpson,0,2,10*2**(n))
    n += 1
    diferencia = abs(ys-y)
    y = ys
print (f'La a integral aproximada (en la iteración {n})  es {y}')
print (f'El error es de {abs(exacto-y).evalf()}')

Aplicando la regla de barrow el valor es 27062.7024138996
La a integral aproximada (en la iteración 6)  es 27062.702480891196
El error es de 0.0000213214760707824


7.- Calcule las fórmulas gaussianas con $2$ y $3$ nodos,en el intervalo $[-1,1]$, siendo la función peso el valor absoluto de la variable. Aplíquelas para aproximar la función $x\; e^x$ en $[-1,1]$ y compare los resultados con el valor exacto (organizando los cálculos de forma adecuada).


In [167]:
#calcula nodos
def gaussiana (a, b, n,w):
    
    #calculamos integrales
    x = sp.symbols("x")
    l = [None]+[sp.integrate(w(x)*x**i,(x, a, b))for i in range(2*n)]
    
    #coeficientes   a resolver
    c = list(sp.symbols('c0:'+ str(n)))
   
    #planteamos ecuaciones del sistema
    s = [np.dot(l[n+i:0+i:-1],c) - l[n+1+i]for i in range(n)]
    r =sp.solve(s,c) #ya tenemos los coeficientes de x**n+ c0x**(n-1) + ... + cix**(n-i)
    #las raíces del polinomio anterior son los nodos buscados
    return sp.solve(-sum([(x**e)*v  for e,v in zip(range(1,-1,-1),r.values())])+x**n)


In [201]:
def w1(x):
    return 1/sp.sqrt(1-x**2)
    #return 1
f=sp.Function('f')
#gauss lendemdre nodos

## coeficientes 
a = -1
b = 1
n = 3
nodos = gaussiana(a,b,n,w1)

c2 = list(sp.symbols('c0:'+ str(n)))
#r =[np.dot([sp.integrate(nodos[j]**i,(x, a, b))for j in range(n)],c2)for i in range(n)]

def w2(y):
    return 1/sp.sqrt(1-y**2)
##
#gaussiana(-1,1,3,w2)
print(nodos)
sp.solve([np.dot([nodos[j]**i for j in range(n)],c2) - sp.integrate( x**i*w(x),(x, a, b)) for i in range(n)],c2)


[6**(1/3)/2, -6**(1/3)/4 - 2**(1/3)*3**(5/6)*I/4, -6**(1/3)/4 + 2**(1/3)*3**(5/6)*I/4]


{c0: 4*6**(1/3)/27 + 2/3,
 c1: -2*6**(1/3)/27 + 2/3 - 2*2**(1/3)*3**(5/6)*I/27,
 c2: -2*6**(1/3)/27 + 2/3 + 2*2**(1/3)*3**(5/6)*I/27}

In [52]:

coefs = list(sp.symbols('c0:'+ str(n)))
coefs

[c0, c1, c2, c3, c4, c5]